In [1]:
import json
import os
from utils import Evaluation, CriteraFeedback, get_structured_feedback, aggregate_feedback, pretty_format_evaluation
from dotenv import load_dotenv
from openai import OpenAI
import instructor

In [5]:
load_dotenv()

openai_client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)

instructor_client = instructor.from_openai(openai_client)

In [2]:
user_ids = ["user1", "user2"]

Store Aggregated feedback for each user

In [ ]:
for user_id in user_ids:
	with open(f"{user_id}-messages.json", "r") as f:
		chat_history = json.load(f)
	
	print(f"Aggregating feedback for user {user_id}")
	
	# Aggregate feedback from chat history for current user
	feedbacks = get_structured_feedback(instructor_client, chat_history)
	aggregated_feedback: Evaluation = aggregate_feedback(feedbacks, openai_client)

	# Store User's aggregated feedback
	with open(f"aggregated-feedback--user-{user_id}.json", "w") as f:
		f.write(aggregated_feedback.model_dump_json(indent=2))
	
	print(f"Stored aggregated feedback for user {user_id}")

In [4]:
student_aggregated_feedbacks = []

for user_id in user_ids:
	with open(f"aggregated-feedback--user-{user_id}.json", "r") as f:
		student_aggregated_feedbacks.append(Evaluation(**json.load(f)))

student_aggregated_feedbacks

[Evaluation(introduction=CriteraFeedback(strengths=['Effectively framed the argument in the introduction and conclusion', 'Clarity in stating the stance on environmental conservation', 'Clear thesis statement'], weaknesses=['Lack of engaging opening statements'], suggestions=['Consider incorporating a more captivating hook to engage readers', 'Incorporate the suggested transition sentence for a smoother transition to the first main point']), structure=CriteraFeedback(strengths=['Clear and logical organization', 'Logical flow of ideas between paragraphs'], weaknesses=['Developing the counterargument section further', 'Adding transition sentences between paragraphs'], suggestions=['Provide more details or statistics in the counterargument section', 'Use transition sentences to enhance the flow', 'Ensure each paragraph has a clear topic sentence and supporting details for improved organization']), argumentation=CriteraFeedback(strengths=['Use of examples to solidify the argument', 'Streng

In [6]:
classroom_aggregated_feedback = aggregate_feedback(openai_client=openai_client, feedbacks=student_aggregated_feedbacks)

with open("classroom-aggregated-feedback.json", "w") as f:
	f.write(classroom_aggregated_feedback.model_dump_json(indent=2))

/Users/13point5/Library/Caches/pypoetry/virtualenvs/classroom-lm-poc-GKspPBel-py3.11/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/13point5/Library/Caches/pypoetry/virtualenvs/classroom-lm-poc-GKspPBel-py3.11/lib/python3.11/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Users/13point5/Library/Caches/pypoetry/virtualenvs/classroom-lm-poc-GKspPBel-py3.11/lib/python3.11/site-packages/numpy/core/_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/13point5/Library/Caches/pypoetry/virtualenvs/classroom-lm-poc-GKspPBel-py3.11/lib/python3.11/site-packages/numpy/core/_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/Users/13point5/Library/Cach

In [7]:
classroom_aggregated_feedback

Evaluation(introduction=CriteraFeedback(strengths=['Effectively framed the argument in the introduction and conclusion', 'Clarity in stating the stance on environmental conservation', 'Clear thesis statement'], weaknesses=['Lack of engaging opening statements'], suggestions=['Consider incorporating a more captivating hook to engage readers', 'Consider adding more factual information and statistics to strengthen the argument.', 'Maintain a balance between conversational tone and formal writing standards.']), structure=CriteraFeedback(strengths=['Clear and logical organization', 'Logical flow of ideas between paragraphs', 'Clear organization of tips for finding statistics and improving sentence variety'], weaknesses=['Developing the counterargument section further', 'Adding transition sentences between paragraphs'], suggestions=['Provide more details or statistics in the counterargument section', 'Use transition sentences to enhance the flow', 'Ensure each paragraph has a clear topic sen

In [8]:
evaluation_str = pretty_format_evaluation(classroom_aggregated_feedback)

In [13]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage

In [28]:
prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(
                "You are a Lesson Plan Generating tool for a grade 8 English classroom."
                "You will be given a query by the teacher along with an aggregate evaluation"
                "of all the students based on their interaction with the Essay Feedback provider tool"
                "Use both pieces of information to personalise your outputs. Use markdown."
				"Only provide the lesson plan, do not regurgitate the evaluation.\n\n"
				f"Class Evaluation: {evaluation_str}"
            )
        )
    ]
)

model = ChatOpenAI()

chain = prompt | model

In [29]:
query = "world war 2"

result = chain.invoke({
	"messages": [
		HumanMessage(
			content=(
				f"Generate a lesson plan to teach this topic: {query}"
			)
		)
	]
})

In [32]:
print(result.content)

Based on the class evaluation and the teacher's query, here is a personalized lesson plan for your grade 8 English class:

## Lesson Plan: Improving Essay Writing Skills

### Objective:
- To enhance students' essay writing skills by focusing on engaging introductions, developing counterarguments, using critical reasoning, incorporating citations, and strengthening conclusions.

### Lesson Outline:

1. **Introduction (15 minutes):**
   - Discuss the importance of engaging opening statements and captivating hooks.
   - Provide examples of effective hooks from various essays.
   - Engage students in a brainstorming session to generate creative hooks for environmental conservation essays.

2. **Structure (20 minutes):**
   - Review the importance of developing the counterargument section and using transition sentences.
   - Analyze sample essays to identify effective transitions and counterargument development.
   - Guide students in rewriting paragraphs with improved transitions and count